# Maschinelle Übersetzung
[Credits to: https://github.com/huggingface/notebooks/blob/main/examples/translation.ipynb]

Vergewissern Sie sich, dass Ihre Version von Transformers mindestens 4.11.0 ist, da die Funktion erst mit dieser Version eingeführt wurde:

In [ ]:
import transformers

print(transformers.__version__)

In [ ]:
import shutil
import os
import pathlib
from pathlib import Path
from pathlib import PurePath

WORKING_PATH=Path(Path(os.getcwd()).parent, Path("data"), Path("translate"))

# shutil.rmtree(WORKING_PATH, ignore_errors=True)
os.makedirs(WORKING_PATH, 0o777, exist_ok=True)

Eine Skriptversion dieses Notizbuchs zur verteilten Feinabstimmung des Modells mit mehreren GPUs oder TPUs finden Sie [hier](https://github.com/huggingface/transformers/tree/master/examples/seq2seq).

# Feinabstimmung eines Modells für eine Übersetzungsaufgabe

In diesem Notizbuch werden wir sehen, wie man ein Modell des [🤗 Transformers](https://github.com/huggingface/transformers) für eine Übersetzungsaufgabe feinabstimmen kann. Wir werden den [WMT-Datensatz](http://www.statmt.org/wmt16/) verwenden, einen maschinellen Übersetzungsdatensatz, der aus einer Sammlung verschiedener Quellen besteht, darunter Nachrichtenkommentare und Parlamentsprotokolle.

In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-ro"

Dieses Notebook ist so aufgebaut, dass es mit jedem model checkpoint  aus dem [Model Hub](https://huggingface.co/models) läuft, solange das Modell eine Sequenz-zu-Sequenz-Version in der Transformers-Bibliothek hat. Hier haben wir den Kontrollpunkt [`Helsinki-NLP/opus-mt-en-ro`](https://huggingface.co/Helsinki-NLP/opus-mt-en-ro) gewählt.

## Laden des Datensatzes

Wir werden die [🤗 Datasets](https://github.com/huggingface/datasets)-Bibliothek verwenden, um die Daten herunterzuladen und die Metrik zu erhalten, die wir für die Auswertung verwenden müssen (um unser Modell mit dem Benchmark zu vergleichen). Dies kann einfach mit den Funktionen `load_dataset` und `load_metric` durchgeführt werden. Wir verwenden hier den englischen/rumänischen Teil des WMT-Datensatzes.

In [ ]:
from datasets import load_dataset
import evaluate
metric = evaluate.load("sacrebleu", trust_remote_code=True)

raw_datasets = load_dataset("wmt16", "ro-en")
metric = evaluate.load("sacrebleu", trust_remote_code=True)

# Was ist Sacrebleu: https://huggingface.co/spaces/evaluate-metric/sacrebleu
# was ist ein bleu score: https://www.geeksforgeeks.org/nlp-bleu-score-for-evaluating-neural-machine-translation-python/

Das `dataset`-Objekt selbst ist [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), das einen Schlüssel für den Trainings-, Validierungs- und Testsatz enthält:

In [ ]:
raw_datasets

Um auf ein tatsächliches Element zuzugreifen, müssen Sie zuerst einen Split auswählen und dann einen Index angeben:

In [ ]:
raw_datasets["train"][0]

Um ein Gefühl dafür zu bekommen, wie die Daten aussehen, zeigt die folgende Funktion einige zufällig ausgewählte Beispiele aus dem Datensatz.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Es können nicht mehr Elemente ausgewählt werden, als im Dataset vorhanden sind."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

Die Metrik ist eine Instanz von [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [ ]:
metric

Sie können die Methode `compute` mit Ihren Vorhersagen und Beschriftungen aufrufen, die eine Liste von dekodierten Strings sein müssen (Liste von Listen für die Beschriftungen):

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

## Preprocessing der Daten

Bevor wir diese Texte in unser Modell einspeisen können, müssen wir sie vorverarbeiten. Dies geschieht durch einen 🤗 Transformers `Tokenizer`, der (wie der Name schon sagt) die Eingaben tokenisiert (einschließlich der Konvertierung der Token in ihre entsprechenden IDs im vortrainierten Vokabular) und in ein Format bringt, das das Modell erwartet, sowie die anderen Eingaben generiert, die das Modell benötigt.

Um all dies zu tun, instanziieren wir unseren Tokenizer mit der Methode `AutoTokenizer.from_pretrained`, die dafür sorgt, dass:

- wir einen Tokenizer erhalten, der der Modellarchitektur entspricht, die wir verwenden wollen,
- wir das Vokabular herunterladen, das beim Vortraining dieses speziellen Prüfpunkts verwendet wurde.

Dieses Vokabular wird zwischengespeichert, so dass es beim nächsten Aufruf der Zelle nicht erneut heruntergeladen werden muss.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Für den mBART-Tokenizer (wie hier) müssen wir die Ausgangs- und die Zielsprache festlegen (damit die Texte richtig vorverarbeitet werden). Sie können die Sprachcodes [hier](https://huggingface.co/facebook/mbart-large-cc25) überprüfen, wenn Sie dieses Notebook für ein anderes Sprachenpaar verwenden.

In [ ]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "ro-RO"

Standardmäßig wird der obige Aufruf einen der schnellen Tokenizer (unterstützt durch Rust) aus der 🤗 Tokenizers-Bibliothek verwenden.

Sie können diesen Tokenizer direkt für einen Satz oder ein Satzpaar aufrufen:

In [ ]:
tokenizer("Hello, this one sentence!")

Je nachdem, welches Modell Sie ausgewählt haben, sehen Sie unterschiedliche Schlüssel im Wörterbuch, das von der obigen Zelle zurückgegeben wird. Sie sind für das, was wir hier tun, nicht von großer Bedeutung (Sie müssen nur wissen, dass sie von dem Modell, das wir später instanziieren werden, benötigt werden). Sie können mehr über sie in [diesem Tutorial] (https://huggingface.co/transformers/preprocessing.html) erfahren, wenn Sie daran interessiert sind.

Anstelle eines Satzes können wir auch eine Liste von Sätzen weitergeben:

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

Um die Ziele für unser Modell vorzubereiten, müssen wir sie innerhalb des Kontextmanagers "as_target_tokenizer" tokenisieren. Dadurch wird sichergestellt, dass der Tokenizer die speziellen Token verwendet, die den Zielen entsprechen:

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

Wenn Sie eine der fünf T5-checkpoints verwenden, die ein spezielles Präfix vor den inputs erfordern, sollten Sie die folgende Zelle anpassen.

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

Wir können dann die Funktion schreiben, die unsere Trainingsbeispiele vorverarbeitet. Wir geben sie einfach mit dem Argument "truncation=True" an den "Tokenizer" weiter. Dadurch wird sichergestellt, dass eine Eingabe, die länger ist als das ausgewählte Modell verarbeiten kann, auf die vom Modell akzeptierte Höchstlänge gekürzt wird. Das Auffüllen wird später (in einem data collator) behandelt, so dass wir die Beispiele auf die längste Länge im batch und nicht auf den gesamten Datensatz auffüllen (pad).

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ro"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Diese Funktion funktioniert mit einem oder mehreren Beispielen. Bei mehreren Beispielen gibt der Tokenizer eine Liste von Listen für jeden Schlüssel zurück:

In [ ]:
preprocess_function(raw_datasets['train'][:2])

Um diese Funktion auf alle Satzpaare in unserem Datensatz anzuwenden, verwenden wir einfach die `map`-Methode unseres `dataset`-Objekts, das wir zuvor erstellt haben. Dadurch wird die Funktion auf alle Elemente aller Splits in `dataset` angewendet, so dass unsere Trainings-, Validierungs- und Testdaten mit einem einzigen Befehl vorverarbeitet werden.

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Noch besser ist, dass die Ergebnisse automatisch von der 🤗 Datasets-Bibliothek zwischengespeichert werden, damit Sie bei der nächsten Ausführung Ihres Notebooks keine Zeit für diesen Schritt aufwenden müssen. Die 🤗 Datasets-Bibliothek ist normalerweise intelligent genug, um zu erkennen, wenn sich die Funktion, die Sie an map übergeben, geändert hat (und daher die Cache-Daten nicht verwendet werden müssen). Sie erkennt zum Beispiel, wenn Sie die Aufgabe in der ersten Zelle ändern und das Notizbuch neu starten. Datasets warnt Sie, wenn es zwischengespeicherte Dateien verwendet. Sie können `load_from_cache_file=False` im Aufruf von `map` übergeben, damit die zwischengespeicherten Dateien nicht verwendet werden und die Vorverarbeitung erneut durchgeführt wird.

Beachten Sie, dass wir `batched=True` übergeben haben, um die Texte stapelweise zusammen zu kodieren. Dies dient dazu, die Vorteile des schnellen Tokenizers, den wir zuvor geladen haben, voll auszunutzen, der Multi-Threading verwendet, um die Texte in einem batch gleichzeitig zu behandeln.

## Fine-tuning des Modells

Nun, da unsere Daten fertig sind, können wir das trainierte Modell herunterladen und feinabstimmen. Da es sich bei unserer Aufgabe um eine Sequenz-zu-Sequenz-Aufgabe handelt, verwenden wir die Klasse `AutoModelForSeq2SeqLM`. Wie beim Tokenizer wird die Methode `from_pretrained` das Modell für uns herunterladen und zwischenspeichern.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Beachten Sie, dass wir keine Warnung wie in unserem Klassifizierungsbeispiel erhalten. Das bedeutet, dass wir alle Gewichte des vorher trainierten Modells verwendet haben und es in diesem Fall keinen zufällig initialisierten Kopf gibt.

Um einen `Seq2SeqTrainer` zu instanziieren, müssen wir drei weitere Dinge definieren. Das Wichtigste ist die Klasse [`Seq2SeqTrainingArguments`] (https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), die alle Attribute enthält, um das Training anzupassen. Sie benötigt einen Ordnernamen, der zum Speichern der Checkpoints des Modells verwendet wird, und alle anderen Argumente sind optional:

In [ ]:
model_name = str(WORKING_PATH).replace("\\","/") + "/studientage_translate"

In [ ]:
import torch
cuda_available = torch.cuda.is_available()
cuda_available

In [ ]:
batch_size = 16

args = Seq2SeqTrainingArguments(
    output_dir=model_name, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=cuda_available,
    push_to_hub=False,
)

Hier stellen wir ein, dass die Auswertung am Ende jeder Epoche erfolgt, passen die Lernrate an, verwenden die oben in der Zelle definierte `batch_size` und passen den weight_decay an. Da der `Seq2SeqTrainer` das Modell regelmäßig speichert und unser Datensatz recht groß ist, weisen wir ihn an, maximal drei Speicherungen vorzunehmen. Schließlich verwenden wir die Option `predict_with_generate` (um Zusammenfassungen korrekt zu generieren) und aktivieren das Training mit gemischter Präzision (um etwas schneller zu sein).

Das letzte Argument, um alles einzurichten, damit wir das Modell während des Trainings regelmäßig an den [Hub](https://huggingface.co/models) senden können. Entfernen Sie es, wenn Sie die Installationsschritte am Anfang des Notizbuchs nicht befolgt haben. Wenn Sie Ihr Modell lokal unter einem anderen Namen als dem des Repositories, in das es gepusht werden soll, speichern wollen, oder wenn Sie Ihr Modell unter einer Organisation und nicht unter Ihrem Namensraum pushen wollen, verwenden Sie das Argument `hub_model_id`, um den Namen des Repositories festzulegen (es muss der vollständige Name sein, einschließlich Ihres Namensraums: zum Beispiel `"sgugger/marian-finetuned-en-to-ro"` oder `"huggingface/marian-finetuned-en-to-ro"`).

Dann brauchen wir eine besondere Art von Datensammler, der nicht nur die Eingaben auf die maximale Länge im Stapel auffüllt, sondern auch die Bezeichnungen:

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
# Was sind Datenkollatoren: https://huggingface.co/docs/transformers/main_classes/data_collator

Das letzte, was wir für unseren `Seq2SeqTrainer` definieren müssen, ist, wie wir die Metrik aus den Vorhersagen berechnen. Wir müssen dafür eine Funktion definieren, die einfach die zuvor geladene `Metrik` verwendet, und wir müssen ein wenig Vorverarbeitung betreiben, um die Vorhersagen in Texte zu dekodieren:

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Dann müssen wir nur noch all dies zusammen mit unseren Datensätzen an den `Seq2SeqTrainer` übergeben:

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Jetzt können wir unser Modell fine-tunen, indem wir einfach die Methode `train` aufrufen:

In [27]:
## Entkommentieren Sie die folgenden 2 Zeilen, um das Modell neu zu trainieren!

# trainer.train()
# model.save_pretrained(model_name)

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.742000,1.290532,28.039200,34.285600


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59542]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59542]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

In [ ]:
from transformers import pipeline
translate = pipeline('translation',model=model_name, tokenizer=tokenizer)

In [ ]:
translate("Hello. I am a machine learning student.")

In [ ]:
translate("Machine learning is great.")